In [9]:
import pandas as pd
import numpy as np
import streamlit as st

In [10]:
brasileirao = pd.read_excel('../data/brasileirao2023.xlsx')
brasileirao

,Temporada,Rodada,Mandante,Score_m,Visitante,Score_v
0,2023,1,PAL,2.0,CUI,1.0
1,2023,1,AME,0.0,FLU,3.0
2,2023,1,CAP,2.0,GOI,0.0
3,2023,1,BOT,2.0,SAO,1.0
4,2023,1,RED,2.0,BAH,1.0
...,...,...,...,...,...,...
375,2023,38,GOI,NaN,AME,NaN
376,2023,38,SAN,NaN,FOR,NaN
377,2023,38,SAO,NaN,FLA,NaN
378,2023,38,VAS,NaN,RED,NaN


In [11]:
brasileirao['Score_m']

0      2.0
1      0.0
2      2.0
3      2.0
4      2.0
      ... 
375    NaN
376    NaN
377    NaN
378    NaN
379    NaN
Name: Score_m, Length: 380, dtype: float64

In [12]:
brasileirao[brasileirao['Score_m'].notnull()]['Rodada'].count()

250

In [13]:
brasileirao = brasileirao[brasileirao['Score_m'].notnull()]
brasileirao

,Temporada,Rodada,Mandante,Score_m,Visitante,Score_v
0,2023,1,PAL,2.0,CUI,1.0
1,2023,1,AME,0.0,FLU,3.0
2,2023,1,CAP,2.0,GOI,0.0
3,2023,1,BOT,2.0,SAO,1.0
4,2023,1,RED,2.0,BAH,1.0
...,...,...,...,...,...,...
245,2023,25,RED,2.0,PAL,1.0
246,2023,25,SAN,4.0,VAS,1.0
247,2023,25,SAO,2.0,COR,1.0
248,2023,25,BOT,1.0,GOI,1.0


In [14]:
times = brasileirao['Mandante'].unique()
tabela = pd.DataFrame()
for time in times:
    vit = emp = der = pro = con = jog = 0
    # emp = 0
    # der = 0
    # pro = 0
    # con = 0
    # jog = 0
    mandante = brasileirao[brasileirao['Mandante'] == time]
    for indice, partida in mandante.iterrows():
        if partida['Score_m'] > partida['Score_v']:
            vit += 1 
        elif partida['Score_m'] == partida['Score_v']:
            emp += 1
        else:
            der += 1
        pro += partida['Score_m']
        con += partida['Score_v']
        jog += 1
    visitante = brasileirao[brasileirao['Visitante'] == time]
    for indice, partida in visitante.iterrows():
        if partida['Score_v'] > partida['Score_m']:
            vit += 1
        elif partida['Score_v'] == partida['Score_m']:
            emp += 1
        else:
            der += 1
        pro += partida['Score_v']
        con += partida['Score_m']
        jog += 1
    new_row = pd.DataFrame({
        '1-Time': [time],
        '3-J' : [jog],
        '4-V' : [vit],
        '5-E' : [emp],
        '6-D' : [der],
        '7-GP': [pro],
        '8-GC': [con]
    })   
    tabela = pd.concat([tabela, new_row], ignore_index = True)
    
tabela['2-P'] = (tabela['4-V'] * 3) + tabela['5-E']
tabela['9-SG'] = tabela['7-GP'] - tabela['8-GC']   
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG
3,BOT,25,16,4,5,40.0,16.0,52,24.0
4,RED,25,12,9,4,35.0,22.0,45,13.0
9,GRE,25,13,5,7,40.0,32.0,44,8.0
0,PAL,25,12,8,5,39.0,20.0,44,19.0
8,FLA,25,12,7,6,37.0,30.0,43,7.0
10,FLU,25,12,5,8,34.0,29.0,41,5.0
6,CAM,25,11,7,7,29.0,19.0,40,10.0
2,CAP,25,11,7,7,37.0,29.0,40,8.0
5,FOR,25,11,6,8,30.0,24.0,39,6.0
12,SAO,25,9,7,9,31.0,27.0,34,4.0


In [15]:
tabela_sort[tabela_sort['7-GP'].notnull().sort_values(ascending=False)]['1-Time'].iloc[0]

C:\Users\samue\AppData\Local\Temp\ipykernel_19820\1631230441.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tabela_sort[tabela_sort['7-GP'].notnull().sort_values(ascending=False)]['1-Time'].iloc[0]


'BOT'

In [16]:
a = brasileirao[brasileirao['Score_m'].notnull()]['Rodada'].unique().max()
a

25

In [17]:
from sklearn.cluster import KMeans
df_data = tabela[[
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG'
]]
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_data)
tabela['cluster'] = kmeans.labels_
# tabela['c_p'] = kmeans.cluster_centers_[kmeans.labels_,0]
# tabela['c_v'] = kmeans.cluster_centers_[kmeans.labels_,1]
# tabela['c_e'] = kmeans.cluster_centers_[kmeans.labels_,2]
# tabela['c_d'] = kmeans.cluster_centers_[kmeans.labels_,3]
# tabela['c_sg'] = kmeans.cluster_centers_[kmeans.labels_,4]
tabela

c:\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
0,PAL,25,12,8,5,39.0,20.0,44,19.0,3
1,AME,25,4,6,15,27.0,51.0,18,-24.0,2
2,CAP,25,11,7,7,37.0,29.0,40,8.0,0
3,BOT,25,16,4,5,40.0,16.0,52,24.0,3
4,RED,25,12,9,4,35.0,22.0,45,13.0,0
5,FOR,25,11,6,8,30.0,24.0,39,6.0,0
6,CAM,25,11,7,7,29.0,19.0,40,10.0,0
7,COR,25,7,9,9,29.0,31.0,30,-2.0,4
8,FLA,25,12,7,6,37.0,30.0,43,7.0,0
9,GRE,25,13,5,7,40.0,32.0,44,8.0,0


In [18]:
print(kmeans.labels_)

[3 2 0 3 0 0 0 4 0 0 0 4 4 4 1 1 1 2 1 1]


In [19]:
kmeans.cluster_centers_

array([[ 41.71428571,  11.71428571,   6.57142857,   6.71428571,
          8.14285714],
       [ 26.8       ,   6.6       ,   7.        ,  11.4       ,
        -10.        ],
       [ 17.5       ,   4.        ,   5.5       ,  15.5       ,
        -25.        ],
       [ 48.        ,  14.        ,   6.        ,   5.        ,
         21.5       ],
       [ 31.5       ,   8.        ,   7.5       ,   9.5       ,
          0.        ]])

In [20]:
df_cluster = pd.DataFrame()
for cluster, colunas in enumerate(kmeans.cluster_centers_):
    print(cluster)
    print(colunas)
    new_row = pd.DataFrame({
        'cluster': [cluster], 
        '2-P': [colunas[0]],              
        '4-V': [colunas[1]],
        '5-E': [colunas[2]],
        '6-D': [colunas[3]],
        '9-SG': [colunas[4]],
    })
    df_cluster = pd.concat([df_cluster, new_row], ignore_index=True)
df_cluster

0
[41.71428571 11.71428571  6.57142857  6.71428571  8.14285714]
1
[ 26.8   6.6   7.   11.4 -10. ]
2
[ 17.5   4.    5.5  15.5 -25. ]
3
[48.  14.   6.   5.  21.5]
4
[31.5  8.   7.5  9.5  0. ]


,cluster,2-P,4-V,5-E,6-D,9-SG
0,0,41.714286,11.714286,6.571429,6.714286,8.142857
1,1,26.800000,6.600000,7.000000,11.400000,-10.000000
2,2,17.500000,4.000000,5.500000,15.500000,-25.000000
3,3,48.000000,14.000000,6.000000,5.000000,21.500000
4,4,31.500000,8.000000,7.500000,9.500000,0.000000


In [21]:
df_cluster_sort = df_cluster.sort_values(by='2-P', ascending=False)
df_cluster_sort

,cluster,2-P,4-V,5-E,6-D,9-SG
3,3,48.000000,14.000000,6.000000,5.000000,21.500000
0,0,41.714286,11.714286,6.571429,6.714286,8.142857
4,4,31.500000,8.000000,7.500000,9.500000,0.000000
1,1,26.800000,6.600000,7.000000,11.400000,-10.000000
2,2,17.500000,4.000000,5.500000,15.500000,-25.000000


In [22]:
df_cluster_sort['grupo'] = ['Título','Libertadores','Sul-Americana','Limbo','Rebaixamento']
df_cluster_sort

,cluster,2-P,4-V,5-E,6-D,9-SG,grupo
3,3,48.000000,14.000000,6.000000,5.000000,21.500000,Título
0,0,41.714286,11.714286,6.571429,6.714286,8.142857,Libertadores
4,4,31.500000,8.000000,7.500000,9.500000,0.000000,Sul-Americana
1,1,26.800000,6.600000,7.000000,11.400000,-10.000000,Limbo
2,2,17.500000,4.000000,5.500000,15.500000,-25.000000,Rebaixamento


In [23]:
df_cluster_grupo = df_cluster_sort[['cluster','grupo']]
df_cluster_grupo

,cluster,grupo
3,3,Título
0,0,Libertadores
4,4,Sul-Americana
1,1,Limbo
2,2,Rebaixamento


In [24]:
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
0,PAL,25,12,8,5,39.0,20.0,44,19.0,3
1,AME,25,4,6,15,27.0,51.0,18,-24.0,2
2,CAP,25,11,7,7,37.0,29.0,40,8.0,0
3,BOT,25,16,4,5,40.0,16.0,52,24.0,3
4,RED,25,12,9,4,35.0,22.0,45,13.0,0
5,FOR,25,11,6,8,30.0,24.0,39,6.0,0
6,CAM,25,11,7,7,29.0,19.0,40,10.0,0
7,COR,25,7,9,9,29.0,31.0,30,-2.0,4
8,FLA,25,12,7,6,37.0,30.0,43,7.0,0
9,GRE,25,13,5,7,40.0,32.0,44,8.0,0


In [25]:
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort.style.background_gradient(cmap='Blues')
# tabela_sort.style.background_gradient(cmap='Blues').format("{:.2f}").render()


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
3,BOT,25,16,4,5,40.000000,16.000000,52,24.000000,3
4,RED,25,12,9,4,35.000000,22.000000,45,13.000000,0
9,GRE,25,13,5,7,40.000000,32.000000,44,8.000000,0
0,PAL,25,12,8,5,39.000000,20.000000,44,19.000000,3
8,FLA,25,12,7,6,37.000000,30.000000,43,7.000000,0
10,FLU,25,12,5,8,34.000000,29.000000,41,5.000000,0
6,CAM,25,11,7,7,29.000000,19.000000,40,10.000000,0
2,CAP,25,11,7,7,37.000000,29.000000,40,8.000000,0
5,FOR,25,11,6,8,30.000000,24.000000,39,6.000000,0
12,SAO,25,9,7,9,31.000000,27.000000,34,4.000000,4


In [26]:
#NORMALIZAR A TABELA E COLORIR...

#grafico com pontuacao original e dos cluster

#grafico dos cluster

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


In [27]:
df_data = tabela[[
    'cluster',              
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG'
]]

In [28]:
X_Train = df_data.drop(columns=['cluster'], axis=1)
X_Test = df_data.drop(columns=['cluster'], axis=1)
y_Train = df_data['cluster']
y_Test = df_data['cluster']

In [29]:
sc_x = StandardScaler()
X_Train = sc_x.fit_transform(X_Train)
X_Test = sc_x.fit_transform(X_Test)

In [30]:
logreg = LogisticRegression(solver="lbfgs", max_iter=500)
logreg.fit(X_Train, y_Train)
pred_logreg = logreg.predict(X_Test)
pred_proba = logreg.predict_proba(X_Test)

In [31]:
tabela["cluster_pred"] = pred_logreg

In [32]:
lista_proba = pred_proba.tolist()

data = []  # Lista para armazenar os dicionários de dados

index = 0
for proba in lista_proba:
    for i in range(0, len(proba)):
        new_row = {"index": index, "prob": i, "valor": round(proba[i], 4)}
        data.append(new_row)
    index += 1

df_prob = pd.DataFrame(data)  # Cria o DataFrame a partir da lista de dicionários
df_prob = df_prob.pivot_table(
    index="index", columns="prob", values="valor", aggfunc="sum"
)
df_prob = df_prob.reset_index()
df_prob = df_prob.set_index("index")


In [33]:
df_prob = df_prob.rename(columns={
    0.0: 'cl_o',
    1.0: 'cl_1',
    2.0: 'cl_2',
    3.0: 'cl_3',
    4.0: 'cl_4'
})

In [34]:
tabela = pd.merge(tabela, df_prob, left_index=True, right_index=True)
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster,cluster_pred,cl_o,cl_1,cl_2,cl_3,cl_4
0,PAL,25,12,8,5,39.0,20.0,44,19.0,3,0,0.5822,0.0100,0.0001,0.3120,0.0958
1,AME,25,4,6,15,27.0,51.0,18,-24.0,2,2,0.0017,0.4020,0.5552,0.0000,0.0411
2,CAP,25,11,7,7,37.0,29.0,40,8.0,0,0,0.6450,0.0534,0.0010,0.1264,0.1742
3,BOT,25,16,4,5,40.0,16.0,52,24.0,3,3,0.4276,0.0004,0.0000,0.5671,0.0050
4,RED,25,12,9,4,35.0,22.0,45,13.0,0,0,0.7034,0.0121,0.0000,0.1985,0.0859
5,FOR,25,11,6,8,30.0,24.0,39,6.0,0,0,0.6390,0.0714,0.0024,0.1114,0.1757
6,CAM,25,11,7,7,29.0,19.0,40,10.0,0,0,0.6231,0.0479,0.0008,0.1476,0.1805
7,COR,25,7,9,9,29.0,31.0,30,-2.0,4,4,0.1210,0.3943,0.0172,0.0069,0.4606
8,FLA,25,12,7,6,37.0,30.0,43,7.0,0,0,0.7480,0.0252,0.0003,0.1381,0.0883
9,GRE,25,13,5,7,40.0,32.0,44,8.0,0,0,0.7521,0.0150,0.0004,0.1829,0.0496


In [35]:
df_pred = tabela[[
    '1-Time',
    'cl_o',
    'cl_1',
    'cl_2',
    'cl_3',
    'cl_4',
]]
df_pred = df_pred.copy()
df_pred

,1-Time,cl_o,cl_1,cl_2,cl_3,cl_4
0,PAL,0.5822,0.0100,0.0001,0.3120,0.0958
1,AME,0.0017,0.4020,0.5552,0.0000,0.0411
2,CAP,0.6450,0.0534,0.0010,0.1264,0.1742
3,BOT,0.4276,0.0004,0.0000,0.5671,0.0050
4,RED,0.7034,0.0121,0.0000,0.1985,0.0859
5,FOR,0.6390,0.0714,0.0024,0.1114,0.1757
6,CAM,0.6231,0.0479,0.0008,0.1476,0.1805
7,COR,0.1210,0.3943,0.0172,0.0069,0.4606
8,FLA,0.7480,0.0252,0.0003,0.1381,0.0883
9,GRE,0.7521,0.0150,0.0004,0.1829,0.0496


In [36]:
df_pred.style.background_gradient(cmap='Blues', axis=1)

,1-Time,cl_o,cl_1,cl_2,cl_3,cl_4
0,PAL,0.582200,0.010000,0.000100,0.312000,0.095800
1,AME,0.001700,0.402000,0.555200,0.000000,0.041100
2,CAP,0.645000,0.053400,0.001000,0.126400,0.174200
3,BOT,0.427600,0.000400,0.000000,0.567100,0.005000
4,RED,0.703400,0.012100,0.000000,0.198500,0.085900
5,FOR,0.639000,0.071400,0.002400,0.111400,0.175700
6,CAM,0.623100,0.047900,0.000800,0.147600,0.180500
7,COR,0.121000,0.394300,0.017200,0.006900,0.460600
8,FLA,0.748000,0.025200,0.000300,0.138100,0.088300
9,GRE,0.752100,0.015000,0.000400,0.182900,0.049600


In [37]:
rodadas = brasileirao['Rodada'].unique()
times = brasileirao['Mandante'].unique()
pontuacao = pd.DataFrame()
data = []  # Lista para armazenar os dicionários de dados

for rodada in rodadas:
    for time in times:
        resultado = brasileirao[(brasileirao['Rodada'] == rodada) & ((brasileirao['Mandante'] == time) | (brasileirao['Visitante'] == time))]
        if len(resultado) > 0:
            resultado = resultado.reset_index()
            if resultado['Mandante'][0] == time:
                if resultado['Score_m'][0] > resultado['Score_v'][0]:
                    pontos = 3
                elif resultado['Score_m'][0] == resultado['Score_v'][0]:
                    pontos = 1
                else:
                    pontos = 0
            else:                  
                if resultado['Score_v'][0] > resultado['Score_m'][0]:
                    pontos = 3
                elif resultado['Score_v'][0] == resultado['Score_m'][0]:
                    pontos = 1
                else:
                    pontos = 0
            new_row = {
                'time': time,
                'rodada': rodada,
                'pontos': pontos
            }        
            data.append(new_row)  # Adiciona o dicionário à lista de dados

pontuacao = pd.DataFrame(data)  # Cria o DataFrame a partir da lista de dicionários

pt_pontuacao = pontuacao.pivot_table(index='rodada', columns='time', values='pontos', aggfunc='sum')
pt_pontuacao_cum = pt_pontuacao.cumsum()
pt_pontuacao_cum

time,AME,BAH,BOT,CAM,CAP,CFC,COR,CRU,CUI,FLA,FLU,FOR,GOI,GRE,INT,PAL,RED,SAN,SAO,VAS
rodada,,,,,,,,,,,,,,,,,,,,
1,0,0,3,0,3,0,3,0,0,3,3,1,0,3,1,3,3,0,0,3
2,0,0,6,1,3,0,3,3,1,3,6,4,3,3,4,4,4,1,3,4
3,0,3,9,4,3,1,3,6,1,3,6,7,3,6,7,7,4,4,4,4
4,0,6,12,4,6,1,4,9,4,3,7,8,3,7,7,10,5,4,7,5
5,1,6,15,7,9,2,4,9,4,6,10,9,3,7,7,13,6,7,8,6
6,1,6,15,10,12,2,5,12,4,9,13,10,6,8,7,14,7,10,9,6
7,4,7,18,13,12,2,5,12,7,12,13,10,7,11,7,15,10,11,12,6
8,4,7,21,14,12,3,8,13,8,13,13,13,7,14,10,16,13,11,15,6
9,7,8,21,17,15,3,8,13,11,16,16,14,7,17,11,19,13,12,15,6


In [38]:
pt_pontuacao_cum = pt_pontuacao_cum.reset_index()
pt_pontuacao_cum

time,rodada,AME,BAH,BOT,CAM,CAP,CFC,COR,CRU,CUI,...,FLU,FOR,GOI,GRE,INT,PAL,RED,SAN,SAO,VAS
0,1,0,0,3,0,3,0,3,0,0,...,3,1,0,3,1,3,3,0,0,3
1,2,0,0,6,1,3,0,3,3,1,...,6,4,3,3,4,4,4,1,3,4
2,3,0,3,9,4,3,1,3,6,1,...,6,7,3,6,7,7,4,4,4,4
3,4,0,6,12,4,6,1,4,9,4,...,7,8,3,7,7,10,5,4,7,5
4,5,1,6,15,7,9,2,4,9,4,...,10,9,3,7,7,13,6,7,8,6
5,6,1,6,15,10,12,2,5,12,4,...,13,10,6,8,7,14,7,10,9,6
6,7,4,7,18,13,12,2,5,12,7,...,13,10,7,11,7,15,10,11,12,6
7,8,4,7,21,14,12,3,8,13,8,...,13,13,7,14,10,16,13,11,15,6
8,9,7,8,21,17,15,3,8,13,11,...,16,14,7,17,11,19,13,12,15,6
9,10,8,9,24,18,16,4,9,14,12,...,17,14,8,17,14,22,14,13,15,6


In [39]:
from sklearn.linear_model import LinearRegression

In [40]:
colunas = pt_pontuacao_cum.columns
df_regressao = pd.DataFrame()
data_regressao = []  # Lista para armazenar os dicionários de resultados de regressão

for coluna in colunas:
    if coluna != 'rodada':
        X = pt_pontuacao_cum['rodada'].values.reshape(-1, 1)
        y = pt_pontuacao_cum[coluna].values.reshape(-1, 1)
        regressor = LinearRegression()
        regressor.fit(X, y)
        A = regressor.intercept_[0]
        B = regressor.coef_[0][0]
        x = A + (B * 38)
        new_row_regressao = {
            'time': coluna,
            'intercept': round(A, 2),
            'slope': round(B, 2),
            'pontuacao_final': round(x, 2)
        }
        data_regressao.append(new_row_regressao)

df_regressao = pd.DataFrame(data_regressao)
df_regressao.sort_values(by='pontuacao_final', ascending=False)

,time,intercept,slope,pontuacao_final
2,BOT,3.20,2.18,85.86
13,GRE,-0.10,1.79,67.89
15,PAL,2.31,1.71,67.23
9,FLA,-0.70,1.78,66.82
16,RED,-1.63,1.76,65.34
10,FLU,1.46,1.60,62.08
4,CAP,0.06,1.59,60.33
11,FOR,1.05,1.44,55.62
3,CAM,0.30,1.45,55.43
18,SAO,1.84,1.33,52.47


In [41]:
df_regressao = df_regressao[['time','slope']]
df_regressao

,time,slope
0,AME,0.74
1,BAH,1.02
2,BOT,2.18
3,CAM,1.45
4,CAP,1.59
5,CFC,0.77
6,COR,1.27
7,CRU,1.12
8,CUI,1.44
9,FLA,1.78


In [42]:
tabela = tabela.merge(df_regressao, left_on='1-Time', right_on='time', how='left')
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster,cluster_pred,cl_o,cl_1,cl_2,cl_3,cl_4,time,slope
0,PAL,25,12,8,5,39.0,20.0,44,19.0,3,0,0.5822,0.0100,0.0001,0.3120,0.0958,PAL,1.71
1,AME,25,4,6,15,27.0,51.0,18,-24.0,2,2,0.0017,0.4020,0.5552,0.0000,0.0411,AME,0.74
2,CAP,25,11,7,7,37.0,29.0,40,8.0,0,0,0.6450,0.0534,0.0010,0.1264,0.1742,CAP,1.59
3,BOT,25,16,4,5,40.0,16.0,52,24.0,3,3,0.4276,0.0004,0.0000,0.5671,0.0050,BOT,2.18
4,RED,25,12,9,4,35.0,22.0,45,13.0,0,0,0.7034,0.0121,0.0000,0.1985,0.0859,RED,1.76
5,FOR,25,11,6,8,30.0,24.0,39,6.0,0,0,0.6390,0.0714,0.0024,0.1114,0.1757,FOR,1.44
6,CAM,25,11,7,7,29.0,19.0,40,10.0,0,0,0.6231,0.0479,0.0008,0.1476,0.1805,CAM,1.45
7,COR,25,7,9,9,29.0,31.0,30,-2.0,4,4,0.1210,0.3943,0.0172,0.0069,0.4606,COR,1.27
8,FLA,25,12,7,6,37.0,30.0,43,7.0,0,0,0.7480,0.0252,0.0003,0.1381,0.0883,FLA,1.78
9,GRE,25,13,5,7,40.0,32.0,44,8.0,0,0,0.7521,0.0150,0.0004,0.1829,0.0496,GRE,1.79


In [43]:
from sklearn.cluster import KMeans
df_data = tabela[[
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG',
    'slope'
]]
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_data)
tabela['cluster'] = kmeans.labels_
tabela['c_p'] = kmeans.cluster_centers_[kmeans.labels_,0]
tabela['c_v'] = kmeans.cluster_centers_[kmeans.labels_,1]
tabela['c_e'] = kmeans.cluster_centers_[kmeans.labels_,2]
tabela['c_d'] = kmeans.cluster_centers_[kmeans.labels_,3]
tabela['c_sg'] = kmeans.cluster_centers_[kmeans.labels_,4]
tabela['c_slope'] = kmeans.cluster_centers_[kmeans.labels_,5]
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort.style.background_gradient(cmap='Blues')


c:\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster,cluster_pred,cl_o,cl_1,cl_2,cl_3,cl_4,time,slope,c_p,c_v,c_e,c_d,c_sg,c_slope
3,BOT,25,16,4,5,40.000000,16.000000,52,24.000000,3,3,0.427600,0.000400,0.000000,0.567100,0.005000,BOT,2.180000,48.000000,14.000000,6.000000,5.000000,21.500000,1.945000
4,RED,25,12,9,4,35.000000,22.000000,45,13.000000,0,0,0.703400,0.012100,0.000000,0.198500,0.085900,RED,1.760000,41.714286,11.714286,6.571429,6.714286,8.142857,1.630000
9,GRE,25,13,5,7,40.000000,32.000000,44,8.000000,0,0,0.752100,0.015000,0.000400,0.182900,0.049600,GRE,1.790000,41.714286,11.714286,6.571429,6.714286,8.142857,1.630000
0,PAL,25,12,8,5,39.000000,20.000000,44,19.000000,3,0,0.582200,0.010000,0.000100,0.312000,0.095800,PAL,1.710000,48.000000,14.000000,6.000000,5.000000,21.500000,1.945000
8,FLA,25,12,7,6,37.000000,30.000000,43,7.000000,0,0,0.748000,0.025200,0.000300,0.138100,0.088300,FLA,1.780000,41.714286,11.714286,6.571429,6.714286,8.142857,1.630000
10,FLU,25,12,5,8,34.000000,29.000000,41,5.000000,0,0,0.731500,0.041700,0.001600,0.129200,0.096000,FLU,1.600000,41.714286,11.714286,6.571429,6.714286,8.142857,1.630000
6,CAM,25,11,7,7,29.000000,19.000000,40,10.000000,0,0,0.623100,0.047900,0.000800,0.147600,0.180500,CAM,1.450000,41.714286,11.714286,6.571429,6.714286,8.142857,1.630000
2,CAP,25,11,7,7,37.000000,29.000000,40,8.000000,0,0,0.645000,0.053400,0.001000,0.126400,0.174200,CAP,1.590000,41.714286,11.714286,6.571429,6.714286,8.142857,1.630000
5,FOR,25,11,6,8,30.000000,24.000000,39,6.000000,0,0,0.639000,0.071400,0.002400,0.111400,0.175700,FOR,1.440000,41.714286,11.714286,6.571429,6.714286,8.142857,1.630000
12,SAO,25,9,7,9,31.000000,27.000000,34,4.000000,4,4,0.341600,0.213800,0.010200,0.042700,0.391700,SAO,1.330000,31.500000,8.000000,7.500000,9.500000,0.000000,1.290000


In [44]:
pontuacao_2 = pt_pontuacao_cum[pt_pontuacao_cum['rodada'] > 19]
pontuacao_2

time,rodada,AME,BAH,BOT,CAM,CAP,CFC,COR,CRU,CUI,...,FLU,FOR,GOI,GRE,INT,PAL,RED,SAN,SAO,VAS
19,20,10,21,48,27,32,14,25,25,28,...,34,29,23,34,24,37,32,21,28,19
20,21,13,21,51,30,33,14,26,25,28,...,35,32,24,37,25,40,35,21,28,19
21,22,16,22,51,31,34,14,27,26,28,...,38,32,25,40,26,41,36,21,31,20
22,23,17,25,51,34,37,14,27,29,29,...,38,35,25,40,29,44,39,21,31,23
23,24,17,25,51,37,40,14,30,29,29,...,41,38,26,43,29,44,42,24,31,26
24,25,18,25,52,40,40,17,30,30,32,...,41,39,27,44,29,44,45,27,34,26


In [45]:
colunas = pontuacao_2.columns
df_regressao = pd.DataFrame()
colunas = pontuacao_2.columns
regressao_data = []

for coluna in colunas:
    if coluna != 'rodada':
        X = pontuacao_2['rodada'].values.reshape(-1, 1)
        y = pontuacao_2[coluna].values.reshape(-1, 1)
        regressor = LinearRegression()
        regressor.fit(X, y)
        A = regressor.intercept_[0]
        B = regressor.coef_[0][0]
        x = A + (B * 38)
        regressao_data.append({
            'time': coluna,
            'intercept': round(A, 2),
            'slope': round(B, 2),
            'pontuacao_final': round(x, 2)
        })

df_regressao = pd.DataFrame(regressao_data)
df_regressao.sort_values(by='pontuacao_final', ascending=False)

,time,intercept,slope,pontuacao_final
16,RED,-19.05,2.54,77.58
3,CAM,-24.05,2.54,72.58
13,GRE,-4.05,1.94,69.78
11,FOR,-11.48,2.03,65.61
4,CAP,-5.14,1.83,64.34
15,PAL,9.52,1.43,63.81
9,FLA,5.24,1.49,61.70
10,FLU,3.76,1.51,61.30
2,BOT,37.81,0.57,59.52
19,VAS,-15.76,1.69,48.30
